##Montagem das bases de dados de contexto fechado
###Rafael Augusto de Souza Silva - Sistemas de Informação - UNIFEI

#Importação das bibliotecas

In [1]:
# importo a biblioteca pandas e a twitter
import pandas as pd
import numpy as np
import tweepy
import nltk
import time
import re
import string
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import json
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
#from datetime import date
from datetime import datetime, date, timezone
from dateutil import parser
from dateutil.relativedelta import relativedelta

#Leitura da base com tweets

In [ ]:
df_bots_tweets = pd.read_csv("tweets.csv", parse_dates=date_cols)
df_bots_tweets.head()

Vizualização das colunas presentes no conjunto com tweets

In [ ]:
df_bots_tweets.columns

#Funções auxiliares

Função que calcula a idade da conta do usuário em dias

In [ ]:
def idade_da_conta(criado_em):
    criacao = criado_em
    #s = criacao
    s = criacao.strftime("%Y-%m-%d %H:%M:%S")
    now = datetime.now()
    t = now.strftime("%Y-%m-%d %H:%M:%S")
    date1 = dt.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    date2 = dt.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    difdate = date2 - date1
    return difdate.days

Função que calcula a média de tweets por dia de cada usuário

In [ ]:
def media_de_tweets(quantidade_tweets,criado_em):  
    media = quantidade_tweets/criado_em
    return media

Função que retorna atributos gerais para compor o conjunto de dados final

In [ ]:
def atributos_gerais(user_id):
    textos = ''
    intervalo_tweets = 0
    dp_intervalo = 0
    dp_dias = 0
    t_segundas = 0
    t_tercas = 0
    t_quartas = 0
    t_quintas = 0
    t_sextas = 0
    t_sabados = 0
    t_domingos = 0
    replies = 0
    mentions = 0
    retweets = 0
    times_retweeted = 0
    tweets = 0
    df_datas = pd.DataFrame()
    datas = []
    favorite_count = 0
    num_hashtags = 0
    num_urls = 0
    i = 0
    df_usuario = df_bots_tweets[df_bots_tweets['user_id']  == user_id]
    if len(df_usuario) > 0:
        for index, row in df_usuario.iterrows():
            if row.in_reply_to_status_id != None: # se o id de resposta for diferente de none
                replies = replies + 1
            mentions = mentions + row.num_mentions        
            count = 0
            text = ''
            rt = str(row.text).find("RT", 0, 3)
            if rt != -1:
                retweets = retweets + 1
            else:
                times_retweeted = times_retweeted + row.retweet_count # se alguem retuitou este tweet
            textos += str(row.text)
            textos += ' | '
            tweets = tweets + 1
            datas.append(row.created_at)
            favorite_count = (favorite_count + row.favorite_count if row.favorite_count != None else favorite_count + 0)
            num_hashtags = (num_hashtags + row.num_hashtags if row.num_hashtags != None else num_hashtags + 0)
            num_urls = (num_urls + row.num_urls if row.num_urls != None else num_urls + 0)

            d = row.created_at
            if d.weekday() == 0:
                t_segundas = t_segundas + 1
            elif d.weekday() == 1:
                t_tercas = t_tercas + 1
            elif d.weekday() == 2:
                t_quartas = t_quartas + 1
            elif d.weekday() == 3:
                t_quintas = t_quintas + 1
            elif d.weekday() == 4:
                t_sextas = t_sextas + 1
            elif d.weekday() == 5:
                t_sabados = t_sabados + 1
            elif d.weekday() == 6:
                t_domingos = t_domingos + 1
        dp = [t_segundas,t_tercas,t_quartas,t_quintas,t_sextas,t_sabados,t_domingos]     
        dp_dias = np.std(dp)
        datas.sort()
        df_datas['datas'] = datas
        dif = df_datas["datas"].diff()
        df_datas['diferenca'] = (df_datas['datas']-df_datas['datas'].shift()).fillna(pd.Timedelta(seconds=0))
        intervalo_tweets = round(df_datas['diferenca'].mean().total_seconds())
        dp_intervalo = round(df_datas['diferenca'].std().total_seconds())
        retorno = {"intervalo_tweets": intervalo_tweets,
                   "dp_intervalo": dp_intervalo,
                   "dp_dias": dp_dias,
                   "t_segundas":t_segundas,
                   "t_tercas": t_tercas,
                   "t_quartas": t_quartas,
                   "t_quintas": t_quintas,
                   "t_sextas": t_sextas,
                   "t_sabados": t_sabados,
                   "t_domingos": t_domingos,
                   "posts": textos,
                   "replies": replies,
                   "mentions": mentions,
                   "retweets": retweets,
                   "times_retweeted": times_retweeted,
                   "tweets": tweets,
                   "favorite_count": favorite_count,
                   "num_hashtags": num_hashtags,
                   "num_urls": num_urls,
                  } 
        return retorno

#Junção dos dados por usuário

In [ ]:
intervalo_tweets = []
dp_intervalo = []
dp_dias = []
t_segundas = []
t_tercas = []
t_quartas = []
t_quintas = []
t_sextas = []
t_sabados = []
t_domingos = []
posts = []
replies = []
mentions = []
retweets = []
times_retweeted = []
tweets = []
favorite_count = []
num_hashtags = []
num_urls = []
average_tweets_per_day = []
account_age = []
idx = []

for index, row in df_bots_users.iterrows():
    df_usuario = df_bots_tweets[df_bots_tweets['user_id']  == row.id]
    if len(df_usuario) > 0:
        retorno = atributos_gerais(row.id)
        intervalo_tweets.append(retorno["intervalo_tweets"])
        dp_intervalo.append(retorno["dp_intervalo"])
        dp_dias.append(retorno["dp_dias"])
        t_segundas.append(retorno["t_segundas"])
        t_tercas.append(retorno["t_tercas"])
        t_quartas.append(retorno["t_quartas"])
        t_quintas.append(retorno["t_quintas"])
        t_sextas.append(retorno["t_sextas"])
        t_sabados.append(retorno["t_sabados"])
        t_domingos.append(retorno["t_domingos"])
        posts.append(retorno["posts"])
        replies.append(retorno["replies"])
        mentions.append(retorno["mentions"])
        retweets.append(retorno["retweets"])
        times_retweeted.append(retorno["times_retweeted"])
        tweets.append(retorno["tweets"])
        favorite_count.append(retorno["favorite_count"])
        num_hashtags.append(retorno["num_hashtags"])
        num_urls.append(retorno["num_urls"])
        average_tweets_per_day.append(media_de_tweets(row.statuses_count,idade_da_conta(row.created_at)))
        account_age.append(idade_da_conta(row.created_at))
    else:
        idx.append(index)
df_bots_users = df_bots_users.drop(labels=idx, axis=0)
df_bots_users = df_bots_users.reset_index()
df_bots_users.drop(columns = "index", inplace=True)

df_bots_users['contagem_favoritos'] = favorite_count # a segunda coluna os seus tweets
df_bots_users['contagem_tweets'] = tweets
df_bots_users['media_tweets_dia'] = average_tweets_per_day 
df_bots_users['idade_da_conta'] = account_age
df_bots_users['n_respostas'] = replies
df_bots_users['n_mencoes'] = mentions
df_bots_users['n_retweets'] = retweets
df_bots_users['n_vezes_retuitado'] = times_retweeted 
df_bots_users['intervalo_tweets'] = intervalo_tweets
df_bots_users['dp_intervalo'] = dp_intervalo
df_bots_users['dp_dias'] = dp_dias
df_bots_users['t_segundas'] = t_segundas
df_bots_users['t_tercas'] = t_tercas
df_bots_users['t_quartas'] = t_quartas
df_bots_users['t_quintas'] = t_quintas
df_bots_users['t_sextas'] = t_sextas
df_bots_users['t_sabados'] = t_sabados
df_bots_users['t_domingos'] = t_domingos
df_bots_users['posts'] = posts
df_bots_users["num_hashtags"] = num_hashtags
df_bots_users["num_urls"] = num_urls

#Exportação do conjunto parcial

In [ ]:
df_bots_users.to_csv('df_bots_spam_3.csv',sep = ';', line_terminator = '', 
                     encoding = 'utf-8', index=False) # no fim exporto um csv com uma base pré-montada

#Leitura da base exportada

In [ ]:
df_bots_users = pd.read_csv("df_bots_spam_3.csv",sep = ';')
lista_bot = df_bots_users.columns

Visualização das colunas do conjunto com bots gerado

In [ ]:
lista_bot

Leitura da base com humanos

In [ ]:
df_humans_users = pd.read_csv("df_userslegitims.csv",sep = ';')
lista_humanos = df_humans_users.columns

Visualização das colunas do conjunto com humanos

In [ ]:
lista_humanos

Verificação de todas as colunas que estão no conjunto de bots e não estão no conjunto de humanos

In [ ]:
[print(x) for x in lista_bot if x not in lista_humanos]

Verificação de todas as clunas que estão no conjunto de humanos e não estão no conjunto de bots

In [ ]:
[print(x) for x in lista_humanos if x not in lista_bot]

Definindo como 0 a classe de humano e 1 a classe de bots

In [ ]:
df_bots_users["tipo_usuario"] = 1
df_humans_users["tipo_usuario"] = 0

Exporto os dois conjuntos

In [ ]:
df_bots_users.to_csv('df_bots_spam_3.csv',sep = ';', line_terminator = '', 
                     encoding = 'utf-8', index=False)

In [ ]:
df_humans_users.to_csv('df_userslegitims.csv',sep = ';', line_terminator = '', 
                     encoding = 'utf-8', index=False) # no fim exporto um csv com uma base pré-montada

#União dos conjuntos de bots e humanos

Leitura dos conjuntos de dados

In [ ]:
df_human = pd.read_csv("df_userslegitims.csv",sep = ';')
df_bots = pd.read_csv("df_bots_spam_3.csv",sep = ';')

Remoção de colunas para que os dois conjuntos fiquem com as colunas iguais

In [ ]:
df_human.drop(columns = ["test_set_1","test_set_2"], inplace=True)
df_bots.drop(columns =["test_set_2"], inplace=True)

Visualização das colunas de cada conjunto

In [ ]:
df_human.columns

In [ ]:
df_bots.columns

Verificação da quantidade de amostras em cada conjunto

In [ ]:
df_human.shape

In [ ]:
df_bots.shape

União dos conjuntos

In [ ]:
df_cresci = pd.concat([df_human,df_bots], ignore_index=True)

Verificação da quantidade de amostras

In [ ]:
df_cresci.shape

Verificação da distribuição de humanos e bots no conjunto unido

In [ ]:
df_cresci.tipo_usuario.value_counts().plot(kind='pie')

Exportação do conjunto unido

In [ ]:
df_cresci.to_csv('df_cresci_3.csv',sep = ';', line_terminator = '', 
                     encoding = 'utf-8', index=False)